In [17]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## for feature slection

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [48]:
df = pd.read_csv('Fraud_std.csv')
df.head(5)

,STATUS,AGE,INCOME,EMP_LENGTH,AMOUNT,INT_RATE,PC_INCOME,HIST_CREDIT,OWNERSHIP_OTHER,OWNERSHIP_OWN,OWNERSHIP_RENT,INTENT_EDUCATION,INTENT_HOMEIMPROVEMENT,INTENT_MEDICAL,INTENT_PERSONAL,INTENT_VENTURE,RATING_B,RATING_C,RATING_D,RATING_E,RATING_F,RATING_G
0,1,-0.921495,-0.114143,5.824575,4.019404,1.355931,3.931411,-0.691554,0,0,1,0,0,0,1,0,0,0,1,0,0,0
1,0,-1.082616,-0.911147,-0.153669,-1.358650,0.264287,-0.657458,-0.938167,0,1,0,1,0,0,0,0,1,0,0,0,0,0
2,1,-0.438130,-0.911147,-0.356321,-0.646849,0.651284,3.744110,-0.691554,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,1,-0.760373,-0.009274,-0.204332,4.019404,1.179210,3.369508,-0.938167,0,0,1,0,0,1,0,0,0,1,0,0,0,0
4,1,-0.599252,-0.188358,-0.001679,4.019404,0.964460,3.556809,-0.444942,0,0,1,0,0,1,0,0,0,1,0,0,0,0


In [3]:
# Define the target variable
target = 'STATUS'

# Define features by excluding the target column
features = [col for col in df.columns if col != target]

# Split the Standard scaled data
X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
feature_sel_model = SelectFromModel(Lasso(alpha=0.005, random_state=0)) # remember to set the seed, the random state in this function
feature_sel_model.fit(X_train, y_train)

SelectFromModel(estimator=Lasso(alpha=0.005, random_state=0))

In [5]:
feature_sel_model.get_support()

array([False,  True,  True,  True,  True,  True, False, False,  True,
        True,  True, False, False, False,  True,  True, False,  True,
        True, False, False])

In [6]:
selected_feature = X_train.columns[(feature_sel_model.get_support())]

# let's print some stats
print('total features: {}'.format((X_train.shape[1])))
print('selected features: {}'.format(len(selected_feature)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(feature_sel_model.estimator_.coef_ == 0)))

total features: 21
selected features: 12
features with coefficients shrank to zero: 9


In [7]:
unselected_feature = X_train.drop(selected_feature, axis=1)

In [8]:
unselected_feature.columns


Index(['AGE', 'HIST_CREDIT', 'OWNERSHIP_OTHER', 'INTENT_HOMEIMPROVEMENT',
       'INTENT_MEDICAL', 'INTENT_PERSONAL', 'RATING_C', 'RATING_F',
       'RATING_G'],
      dtype='object')

In [9]:
#from sklearn.decomposition import PCA

In [10]:
'''pca=PCA(n_components=2)
pca.fit(unselected_feature)
x_pca=pca.transform(unselected_feature)'''

'pca=PCA(n_components=2)\npca.fit(unselected_feature)\nx_pca=pca.transform(unselected_feature)'

In [11]:
'''col_name_pca = ['PCA_1','PCA_2']
x_pca = pd.DataFrame(x_pca, columns = col_name_pca)'''

"col_name_pca = ['PCA_1','PCA_2']\nx_pca = pd.DataFrame(x_pca, columns = col_name_pca)"

Logistic Regression Model

In [14]:
from sklearn.linear_model import LogisticRegression
classifier=LogisticRegression()

In [26]:
from sklearn.model_selection import GridSearchCV
parameter={'penalty':['l1','l2','elasticnet'],
           'C':[1,2,3,4,5,6,10,20,30,40,50],
           'max_iter':[100,200,300]}

In [27]:
classifier_regressor=GridSearchCV(classifier,param_grid=parameter,scoring='accuracy',cv=5)

In [28]:
classifier_regressor.fit(X_train[selected_feature],y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [1, 2, 3, 4, 5, 6, 10, 20, 30, 40, 50],
                         'max_iter': [100, 200, 300],
                         'penalty': ['l1', 'l2', 'elasticnet'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='accuracy')

In [29]:
print(classifier_regressor.best_params_)

{'C': 1, 'max_iter': 200, 'penalty': 'l1', 'solver': 'saga'}


In [30]:
print(classifier_regressor.best_score_)

0.8634514428257238


In [31]:
##prediction
y_pred=classifier_regressor.predict(X_test[selected_feature])

In [32]:
from sklearn.metrics import accuracy_score,classification_report

In [35]:
score_logistic=accuracy_score(y_pred,y_test)
print(score)

0.8586773055086696


In [34]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.96      0.87      0.91      5545
           1       0.52      0.77      0.62       972

    accuracy                           0.86      6517
   macro avg       0.74      0.82      0.77      6517
weighted avg       0.89      0.86      0.87      6517



SVM

In [37]:
from sklearn.svm import SVC

# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Create a SVM model
svm = SVC()

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train[selected_feature],y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

Fitting 5 folds for each of 32 candidates, totalling 160 fits
Best parameters: {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Best cross-validation score: 0.9035067520407317


In [38]:
X_train = X_train[selected_feature]
X_test = X_test[selected_feature]

In [39]:
from sklearn.tree import DecisionTreeClassifier
# Define the parameter grid
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}

# Create a Decision Tree model
decision_tree = DecisionTreeClassifier()

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 864 candidates, totalling 4320 fits
Best parameters: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}
Best cross-validation score: 0.9106813814392527
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      5072
           1       0.86      0.68      0.76      1445

    accuracy                           0.91      6517
   macro avg       0.89      0.83      0.85      6517
weighted avg       0.90      0.91      0.90      6517



In [43]:
from sklearn.ensemble import RandomForestClassifier
# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Create a Random Forest model
random_forest = RandomForestClassifier()

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best parameters: {'bootstrap': True, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.9160144240614592
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      5072
           1       0.89      0.69      0.78      1445

    accuracy                           0.91      6517
   macro avg       0.90      0.83      0.86      6517
weighted avg       0.91      0.91      0.91      6517



In [44]:
from sklearn.ensemble import AdaBoostClassifier

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 1, 10],
    'base_estimator__max_depth': [1, 2, 3, 4, 5]
}

# Create an AdaBoost model with DecisionTreeClassifier as base estimator
base_estimator = DecisionTreeClassifier()
adaboost = AdaBoostClassifier(base_estimator=base_estimator)

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=adaboost, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 60 candidates, totalling 300 fits
Best parameters: {'base_estimator__max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 200}
Best cross-validation score: 0.9180477947936699
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5072
           1       0.92      0.70      0.79      1445

    accuracy                           0.92      6517
   macro avg       0.92      0.84      0.87      6517
weighted avg       0.92      0.92      0.92      6517



In [46]:
from sklearn.ensemble import GradientBoostingClassifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Create a Gradient Boosting model
gradient_boosting = GradientBoostingClassifier()

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=gradient_boosting, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
Best parameters: {'learning_rate': 0.2, 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Best cross-validation score: 0.9183547639549806
              precision    recall  f1-score   support

           0       0.92      0.97      0.95      5072
           1       0.89      0.71      0.79      1445

    accuracy                           0.91      6517
   macro avg       0.90      0.84      0.87      6517
weighted avg       0.91      0.91      0.91      6517



In [47]:
from sklearn.naive_bayes import GaussianNB

# Define the parameter grid
param_grid = {
    'var_smoothing': np.logspace(0, -9, num=100)
}

# Create a GaussianNB model
naive_bayes = GaussianNB()

# Set up the GridSearchCV
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)

# Train the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_}")

# Predict on the test set
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'var_smoothing': 0.08111308307896872}
Best cross-validation score: 0.8474525063455653
              precision    recall  f1-score   support

           0       0.89      0.91      0.90      5072
           1       0.66      0.62      0.64      1445

    accuracy                           0.85      6517
   macro avg       0.78      0.77      0.77      6517
weighted avg       0.84      0.85      0.84      6517

